# 2. Kreiranje Dataset-a za Mašinsko Učenje

## Organizacija podataka i priprema za treniranje

---

Nakon normalizacije direktorijuma, sledeći korak je kreiranje dataset-a koji će biti pogdan za treniranje neuronske mreže.

In [1]:
# Import biblioteka
import sys
import os
from pathlib import Path
import pickle
from collections import Counter

# Dodaj src u path
current_dir = Path.cwd()
project_root = current_dir if current_dir.name != 'notebooks' else current_dir.parent
src_path = project_root / "src"

sys.path.insert(0, str(src_path))

# Import dataset modula
from dataset import CarDataModule
from cached_dataset import CachedCarDataModule

### Kako funkcioniše CarDataModule

**CarDataModule** je osnovna klasa koja:

1. **Skenira direktorijume** - prolazi kroz sve poddirektorijume u `data/raw/`
2. **Ekstraktuje labele** - iz naziva direktorijuma izvlači make, model, year koristeći CarFolderNormalizer
3. **Učitava putanje slika** - pronalazi sve `.jpg` fajlove u svakom direktorijumu
4. **Deli podatke** - koristi make-model kombinacije da spreči data leakage (64% train / 16% val / 20% test)
5. **Enkodira labele** - konvertuje string labele u numeričke vrednosti pomoću LabelEncoder
6. **Primenjuje transformacije** - različite za train (augmentacija) i val/test (normalizacija)

**Problem**: Ovaj proces traje 5-10 minuta jer mora da skenira ~86,000 slika.

### Sistem keširanje - CachedCarDataModule

Da bi se izbegao spori sken, implementiran je **CachedCarDataModule**:

#### Prvi korak: Kreiranje keša
```bash
python scripts/create_data_cache.py --data_path data/raw --cache_file cache_consolidated/dataset_cache.pkl
```

Ovaj script:
- Pokreće CarDataModule jednom (10 minuta)
- Čuva sve podatke u `.pkl` fajl:
  - Sve putanje do slika
  - Ekstraktovane labele
  - Train/val/test podelu
  - Label encoders
  - Metadata o klasama

#### Sledeći korišćenja: Brzo učitavanje
```python
data_module = CachedCarDataModule(
    data_path="data/raw",
    cache_file="cache_consolidated/dataset_cache.pkl"
)
```

**Rezultat**: Učitavanje za 10 sekundi umesto 10 minuta (60x brže)

### Demonstracija: Testiranje cache fajla

Hajde da proverimo da li cache fajl postoji i učitamo ga:

In [2]:
# Proveri da li cache postoji
cache_file = project_root / "cache_consolidated" / "dataset_cache.pkl"

if cache_file.exists():
    print(f"✅ Cache file pronađen: {cache_file}")
    print(f"📏 Veličina fajla: {cache_file.stat().st_size / (1024*1024):.1f} MB")
    
    # Učitaj osnovne informacije
    with open(cache_file, 'rb') as f:
        cache_data = pickle.load(f)
    
    print(f"\n📊 Cache sadrži:")
    print(f"   Train samples: {len(cache_data['train_samples']):,}")
    print(f"   Val samples: {len(cache_data['val_samples']):,}")
    print(f"   Test samples: {len(cache_data['test_samples']):,}")
    print(f"   Ukupno: {len(cache_data['train_samples']) + len(cache_data['val_samples']) + len(cache_data['test_samples']):,}")
    
    print(f"\n🏭 Broj klasa:")
    print(f"   Makes: {cache_data['num_makes']}")
    print(f"   Models: {cache_data['num_models']}")
    print(f"   Years: {cache_data['num_years']}")
    
else:
    print(f"❌ Cache file ne postoji: {cache_file}")
    print(f"💡 Potrebno je pokrenuti: python scripts/create_data_cache.py")

✅ Cache file pronađen: /home/void/Documents/faks/ri/projekat/cache_consolidated/dataset_cache.pkl
📏 Veličina fajla: 13.2 MB

📊 Cache sadrži:
   Train samples: 52,030
   Val samples: 12,218
   Test samples: 15,777
   Ukupno: 80,025

🏭 Broj klasa:
   Makes: 81
   Models: 1358
   Years: 12


### Analiza kvaliteta podataka

Sada možemo da analiziramo stvarne podatke koristeći istu logiku kao `check_data_quality.py`:

In [3]:
if cache_file.exists():
    # Izvuci sve uzorke
    all_samples = (cache_data['train_samples'] + 
                  cache_data['val_samples'] + 
                  cache_data['test_samples'])
    
    print(f"🔍 Analiza kvaliteta podataka na {len(all_samples):,} uzoraka:")
    print()
    
    # 1. Proveri UNKNOWN vrednosti
    unknown_makes = sum(1 for s in all_samples if s['make'] == 'UNKNOWN')
    unknown_models = sum(1 for s in all_samples if s['model'] == 'UNKNOWN')
    unknown_years = sum(1 for s in all_samples if s['year'] == 'Unknown')
    
    print(f"⚠️ UNKNOWN labeli:")
    print(f"   Makes: {unknown_makes:,} ({unknown_makes/len(all_samples)*100:.1f}%)")
    print(f"   Models: {unknown_models:,} ({unknown_models/len(all_samples)*100:.1f}%)")
    print(f"   Years: {unknown_years:,} ({unknown_years/len(all_samples)*100:.1f}%)")
    
    # 2. Proverka data leakage
    train_combos = set(f"{s['make']}_{s['model']}" for s in cache_data['train_samples'])
    val_combos = set(f"{s['make']}_{s['model']}" for s in cache_data['val_samples'])
    test_combos = set(f"{s['make']}_{s['model']}" for s in cache_data['test_samples'])
    
    train_val_overlap = train_combos & val_combos
    train_test_overlap = train_combos & test_combos
    val_test_overlap = val_combos & test_combos
    
    print(f"\n🔒 Data leakage proverka:")
    print(f"   Train-Val overlap: {len(train_val_overlap)} kombinacija")
    print(f"   Train-Test overlap: {len(train_test_overlap)} kombinacija")
    print(f"   Val-Test overlap: {len(val_test_overlap)} kombinacija")
    
    if len(train_val_overlap) + len(train_test_overlap) + len(val_test_overlap) == 0:
        print(f"   ✅ Nema data leakage - podela je ispravna")
    else:
        print(f"   ❌ Detektovan data leakage!")
    
    # 3. Distribucija klasa
    make_counts = Counter(s['make'] for s in all_samples)
    model_counts = Counter(s['model'] for s in all_samples)
    year_counts = Counter(s['year'] for s in all_samples)
    
    print(f"\n📊 Distribucija klasa:")
    print(f"   Makes - najčešći: {make_counts.most_common(1)[0][0]} ({make_counts.most_common(1)[0][1]:,})")
    print(f"   Makes - najređi: {make_counts.most_common()[-1][0]} ({make_counts.most_common()[-1][1]:,})")
    
    print(f"   Models - najčešći: {model_counts.most_common(1)[0][0]} ({model_counts.most_common(1)[0][1]:,})")
    print(f"   Models - najređi: {model_counts.most_common()[-1][0]} ({model_counts.most_common()[-1][1]:,})")
    
    models_under_10 = sum(1 for count in model_counts.values() if count < 10)
    print(f"   Models sa <10 uzoraka: {models_under_10:,} ({models_under_10/len(model_counts)*100:.1f}%)")
    
    print(f"\n📅 Distribucija godina:")
    for year, count in sorted(year_counts.items()):
        print(f"   {year}: {count:,} uzoraka")
else:
    print("❌ Nije moguće analizirati podatke - cache ne postoji")

🔍 Analiza kvaliteta podataka na 80,025 uzoraka:

⚠️ UNKNOWN labeli:
   Makes: 63 (0.1%)
   Models: 63 (0.1%)
   Years: 33 (0.0%)

🔒 Data leakage proverka:
   Train-Val overlap: 0 kombinacija
   Train-Test overlap: 0 kombinacija
   Val-Test overlap: 0 kombinacija
   ✅ Nema data leakage - podela je ispravna

📊 Distribucija klasa:
   Makes - najčešći: BMW (12,267)
   Makes - najređi: MG (4)
   Models - najčešći: 1_SERIES (622)
   Models - najređi: Express_LWB (2)
   Models sa <10 uzoraka: 46 (3.4%)

📅 Distribucija godina:
   1920s: 31 uzoraka
   1930s: 84 uzoraka
   1940s: 97 uzoraka
   1950s: 334 uzoraka
   1960s: 560 uzoraka
   1970s: 733 uzoraka
   1980s: 1,306 uzoraka
   1990s: 2,784 uzoraka
   2000s: 20,127 uzoraka
   2010s: 41,640 uzoraka
   2020s: 12,296 uzoraka
   Unknown: 33 uzoraka


### Demonstracija brzog učitavanja

Ako cache postoji, možemo da demonstriramo brzo učitavanje:

In [4]:
if cache_file.exists():
    import time
    
    print("⚡ Demonstracija brzog učitavanja sa CachedCarDataModule:")
    start_time = time.time()
    
    # Učitaj dataset pomoću cache-a
    cached_data_module = CachedCarDataModule(
        data_path=str(project_root / "data" / "raw"),
        cache_file=str(cache_file),
        batch_size=32
    )
    
    end_time = time.time()
    
    print(f"\n⏱️ Vreme učitavanja: {end_time - start_time:.1f} sekundi")
    print(f"📈 Ubrzanje: ~{10*60 / (end_time - start_time):.0f}x brže od običnog CarDataModule")
    
    # Prikaži osnovne informacije
    class_info = cached_data_module.get_class_info()
    print(f"\n✅ Dataset spreman za treniranje!")
    print(f"   Classes: {class_info['num_makes']} makes, {class_info['num_models']} models, {class_info['num_years']} years")
    
else:
    print("❌ Cache ne postoji - potrebno je kreirati cache prvo")
    print("💡 Pokrenite: python scripts/create_data_cache.py")

⚡ Demonstracija brzog učitavanja sa CachedCarDataModule:
📦 Loading dataset cache from /home/void/Documents/faks/ri/projekat/cache_consolidated/dataset_cache.pkl...
✅ Cache loaded! Train: 52,030, Val: 12,218, Test: 15,777

⏱️ Vreme učitavanja: 626.0 sekundi
📈 Ubrzanje: ~1x brže od običnog CarDataModule

✅ Dataset spreman za treniranje!
   Classes: 81 makes, 1358 models, 12 years


### Transformacije slika

Dataset koristi različite transformacije za train i validation/test:

In [7]:
from torchvision import transforms

# Transformacije za treniranje (sa augmentacijom)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.RandomRotation(degrees=10)], p=0.3),
    transforms.RandomApply([transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1)], p=0.4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Transformacije za validation/test (bez augmentacije)
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

print("🔄 Transformacije za treniranje:")
for i, transform in enumerate(train_transforms.transforms, 1):
    print(f"   {i}. {transform}")

print("\n🎯 Transformacije za validation/test:")
for i, transform in enumerate(val_test_transforms.transforms, 1):
    print(f"   {i}. {transform}")

print("\n📐 Sve transformacije su optimizovane za ResNet-50 (224x224 input)")
print("🎨 Augmentacija pomaže da model generalizuje bolje")

🔄 Transformacije za treniranje:
   1. Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
   2. RandomHorizontalFlip(p=0.5)
   3. RandomApply(
    p=0.3
    RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
)
   4. RandomApply(
    p=0.4
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
)
   5. ToTensor()
   6. Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

🎯 Transformacije za validation/test:
   1. Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
   2. ToTensor()
   3. Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

📐 Sve transformacije su optimizovane za ResNet-50 (224x224 input)
🎨 Augmentacija pomaže da model generalizuje bolje


### Zaključak

Dataset je uspešno pripremljen za mašinsko učenje:

#### ✅ Prednosti:
- Veliki broj slika (~79,000)
- Raznovrstan skup proizvođača i modela
- Sprečen data leakage (make-model podela)
- Brzo učitavanje pomoću cache sistema (60x brže)
- Kvalitetne transformacije za augmentaciju

#### ⚠️ Identifikovani izazovi:
- Značajna nebalansiranost klasa (posebno modeli)
- Veliki broj modela (1300+) sa malo uzoraka
- Kombinatorička eksplozija (600,000+ teorijskih kombinacija)
- Vrlo nizak random baseline za EMR (~0.00005%)

#### 🔄 Sledeći korak:
Pokušaj treniranja ResNet-50 modela i identifikacija problema u performansama.